# Your Title Here

**Name(s)**: (your name(s) here)

**Website Link**: (your website link)

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path

import plotly.express as px
pd.options.plotting.backend = 'plotly'

# from dsc80_utils import * # Feel free to uncomment and use this.

## Step 1: Introduction

In [2]:
# TODO

## Step 2: Data Cleaning and Exploratory Data Analysis

In [3]:
!pip install openpyxl

In [4]:
# TODO
df = pd.read_excel('outage.xlsx',skiprows=5)
df

,variables,OBS,YEAR,MONTH,U.S._STATE,POSTAL.CODE,NERC.REGION,CLIMATE.REGION,ANOMALY.LEVEL,CLIMATE.CATEGORY,...,POPPCT_URBAN,POPPCT_UC,POPDEN_URBAN,POPDEN_UC,POPDEN_RURAL,AREAPCT_URBAN,AREAPCT_UC,PCT_LAND,PCT_WATER_TOT,PCT_WATER_INLAND
0,Units,NaN,NaN,NaN,NaN,NaN,NaN,NaN,numeric,NaN,...,%,%,persons per square mile,persons per square mile,persons per square mile,%,%,%,%,%
1,NaN,1.0,2011.0,7.0,Minnesota,MN,MRO,East North Central,-0.3,normal,...,73.27,15.28,2279,1700.5,18.2,2.14,0.6,91.592666,8.407334,5.478743
2,NaN,2.0,2014.0,5.0,Minnesota,MN,MRO,East North Central,-0.1,normal,...,73.27,15.28,2279,1700.5,18.2,2.14,0.6,91.592666,8.407334,5.478743
3,NaN,3.0,2010.0,10.0,Minnesota,MN,MRO,East North Central,-1.5,cold,...,73.27,15.28,2279,1700.5,18.2,2.14,0.6,91.592666,8.407334,5.478743
4,NaN,4.0,2012.0,6.0,Minnesota,MN,MRO,East North Central,-0.1,normal,...,73.27,15.28,2279,1700.5,18.2,2.14,0.6,91.592666,8.407334,5.478743
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1530,NaN,1530.0,2011.0,12.0,North Dakota,ND,MRO,West North Central,-0.9,cold,...,59.9,19.9,2192.2,1868.2,3.9,0.27,0.1,97.599649,2.401765,2.401765
1531,NaN,1531.0,2006.0,NaN,North Dakota,ND,MRO,West North Central,NaN,NaN,...,59.9,19.9,2192.2,1868.2,3.9,0.27,0.1,97.599649,2.401765,2.401765
1532,NaN,1532.0,2009.0,8.0,South Dakota,SD,RFC,West North Central,0.5,warm,...,56.65,26.73,2038.3,1905.4,4.7,0.3,0.15,98.307744,1.692256,1.692256
1533,NaN,1533.0,2009.0,8.0,South Dakota,SD,MRO,West North Central,0.5,warm,...,56.65,26.73,2038.3,1905.4,4.7,0.3,0.15,98.307744,1.692256,1.692256


In [5]:
df_no_unit = df.drop(index=0).drop(columns=["variables"])
df_no_unit

,OBS,YEAR,MONTH,U.S._STATE,POSTAL.CODE,NERC.REGION,CLIMATE.REGION,ANOMALY.LEVEL,CLIMATE.CATEGORY,OUTAGE.START.DATE,...,POPPCT_URBAN,POPPCT_UC,POPDEN_URBAN,POPDEN_UC,POPDEN_RURAL,AREAPCT_URBAN,AREAPCT_UC,PCT_LAND,PCT_WATER_TOT,PCT_WATER_INLAND
1,1.0,2011.0,7.0,Minnesota,MN,MRO,East North Central,-0.3,normal,2011-07-01 00:00:00,...,73.27,15.28,2279,1700.5,18.2,2.14,0.6,91.592666,8.407334,5.478743
2,2.0,2014.0,5.0,Minnesota,MN,MRO,East North Central,-0.1,normal,2014-05-11 00:00:00,...,73.27,15.28,2279,1700.5,18.2,2.14,0.6,91.592666,8.407334,5.478743
3,3.0,2010.0,10.0,Minnesota,MN,MRO,East North Central,-1.5,cold,2010-10-26 00:00:00,...,73.27,15.28,2279,1700.5,18.2,2.14,0.6,91.592666,8.407334,5.478743
4,4.0,2012.0,6.0,Minnesota,MN,MRO,East North Central,-0.1,normal,2012-06-19 00:00:00,...,73.27,15.28,2279,1700.5,18.2,2.14,0.6,91.592666,8.407334,5.478743
5,5.0,2015.0,7.0,Minnesota,MN,MRO,East North Central,1.2,warm,2015-07-18 00:00:00,...,73.27,15.28,2279,1700.5,18.2,2.14,0.6,91.592666,8.407334,5.478743
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1530,1530.0,2011.0,12.0,North Dakota,ND,MRO,West North Central,-0.9,cold,2011-12-06 00:00:00,...,59.9,19.9,2192.2,1868.2,3.9,0.27,0.1,97.599649,2.401765,2.401765
1531,1531.0,2006.0,NaN,North Dakota,ND,MRO,West North Central,NaN,NaN,NaN,...,59.9,19.9,2192.2,1868.2,3.9,0.27,0.1,97.599649,2.401765,2.401765
1532,1532.0,2009.0,8.0,South Dakota,SD,RFC,West North Central,0.5,warm,2009-08-29 00:00:00,...,56.65,26.73,2038.3,1905.4,4.7,0.3,0.15,98.307744,1.692256,1.692256
1533,1533.0,2009.0,8.0,South Dakota,SD,MRO,West North Central,0.5,warm,2009-08-29 00:00:00,...,56.65,26.73,2038.3,1905.4,4.7,0.3,0.15,98.307744,1.692256,1.692256


In [6]:
df_no_unit.columns

Index(['OBS', 'YEAR', 'MONTH', 'U.S._STATE', 'POSTAL.CODE', 'NERC.REGION',
       'CLIMATE.REGION', 'ANOMALY.LEVEL', 'CLIMATE.CATEGORY',
       'OUTAGE.START.DATE', 'OUTAGE.START.TIME', 'OUTAGE.RESTORATION.DATE',
       'OUTAGE.RESTORATION.TIME', 'CAUSE.CATEGORY', 'CAUSE.CATEGORY.DETAIL',
       'HURRICANE.NAMES', 'OUTAGE.DURATION', 'DEMAND.LOSS.MW',
       'CUSTOMERS.AFFECTED', 'RES.PRICE', 'COM.PRICE', 'IND.PRICE',
       'TOTAL.PRICE', 'RES.SALES', 'COM.SALES', 'IND.SALES', 'TOTAL.SALES',
       'RES.PERCEN', 'COM.PERCEN', 'IND.PERCEN', 'RES.CUSTOMERS',
       'COM.CUSTOMERS', 'IND.CUSTOMERS', 'TOTAL.CUSTOMERS', 'RES.CUST.PCT',
       'COM.CUST.PCT', 'IND.CUST.PCT', 'PC.REALGSP.STATE', 'PC.REALGSP.USA',
       'PC.REALGSP.REL', 'PC.REALGSP.CHANGE', 'UTIL.REALGSP', 'TOTAL.REALGSP',
       'UTIL.CONTRI', 'PI.UTIL.OFUSA', 'POPULATION', 'POPPCT_URBAN',
       'POPPCT_UC', 'POPDEN_URBAN', 'POPDEN_UC', 'POPDEN_RURAL',
       'AREAPCT_URBAN', 'AREAPCT_UC', 'PCT_LAND', 'PCT_WATER_TOT',
    

In [7]:
df_cleaned = df_no_unit.get(['YEAR', 'MONTH','OUTAGE.START.TIME','U.S._STATE', 'CAUSE.CATEGORY', 'CAUSE.CATEGORY.DETAIL','CUSTOMERS.AFFECTED', 'OUTAGE.DURATION', 'DEMAND.LOSS.MW','POPULATION', 'TOTAL.SALES','TOTAL.CUSTOMERS', 'PC.REALGSP.CHANGE'])
df_cleaned.isna().sum(axis=1)
df_cleaned.loc[:, "OUTAGE.DURATION"] = pd.to_numeric(df_cleaned["OUTAGE.DURATION"], errors='coerce')
df_cleaned

,YEAR,MONTH,OUTAGE.START.TIME,U.S._STATE,CAUSE.CATEGORY,CAUSE.CATEGORY.DETAIL,CUSTOMERS.AFFECTED,OUTAGE.DURATION,DEMAND.LOSS.MW,POPULATION,TOTAL.SALES,TOTAL.CUSTOMERS,PC.REALGSP.CHANGE
1,2011.0,7.0,17:00:00,Minnesota,severe weather,NaN,70000.0,3060.0,NaN,5348119.0,6562520,2595696.0,1.6
2,2014.0,5.0,18:38:00,Minnesota,intentional attack,vandalism,NaN,1.0,NaN,5457125.0,5284231,2640737.0,1.9
3,2010.0,10.0,20:00:00,Minnesota,severe weather,heavy wind,70000.0,3000.0,NaN,5310903.0,5222116,2586905.0,2.7
4,2012.0,6.0,04:30:00,Minnesota,severe weather,thunderstorm,68200.0,2550.0,NaN,5380443.0,5787064,2606813.0,0.6
5,2015.0,7.0,02:00:00,Minnesota,severe weather,NaN,250000.0,1740.0,250,5489594.0,5970339,2673531.0,1.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1530,2011.0,12.0,08:00:00,North Dakota,public appeal,NaN,34500.0,720.0,155,685326.0,1313678,394394.0,9.8
1531,2006.0,NaN,NaN,North Dakota,fuel supply emergency,Coal,NaN,NaN,1650,649422.0,NaN,366037.0,3.5
1532,2009.0,8.0,22:54:00,South Dakota,islanding,NaN,NaN,59.0,84,807067.0,924051,436229.0,0
1533,2009.0,8.0,11:00:00,South Dakota,islanding,NaN,NaN,181.0,373,807067.0,924051,436229.0,0


In [8]:
fig = px.histogram(df_no_unit, x='CAUSE.CATEGORY', nbins=30, title='Histogram of Cause of Power Outage')
fig.show()


In [9]:
fig2 = px.histogram(df_no_unit, x='CLIMATE.CATEGORY', nbins=30, title='Histogram of Climate of Power Outage')
fig2.show()

In [10]:
# Boxplot for OUTAGE.DURATION

fig2 = px.box(df, y='OUTAGE.DURATION', title='Outage Duration Distribution')
fig2.update_layout(yaxis_title='Duration (hours)')
fig2.show()

In [11]:
df_new = df_cleaned.copy()
df_new['OUTAGE.DURATION'] = df_new['OUTAGE.DURATION'].clip(upper=7020)

fig2 = px.box(df_new, y='OUTAGE.DURATION', title='Outage Duration Distribution (Cleaned)')
fig2.update_layout(yaxis_title='Duration (hours)')
fig2.show()

In [12]:
df_new = df_cleaned.copy()
numeric_cols = ["OUTAGE.DURATION"]

# Use IQR to identify outliers and remove these outliers
for col in numeric_cols:
    Q1 = df_new[col].quantile(0.25)
    Q3 = df_new[col].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
 
    df_new = df_new[(df_new[col] >= lower) & (df_new[col] <= upper)]
df_new

,YEAR,MONTH,OUTAGE.START.TIME,U.S._STATE,CAUSE.CATEGORY,CAUSE.CATEGORY.DETAIL,CUSTOMERS.AFFECTED,OUTAGE.DURATION,DEMAND.LOSS.MW,POPULATION,TOTAL.SALES,TOTAL.CUSTOMERS,PC.REALGSP.CHANGE
1,2011.0,7.0,17:00:00,Minnesota,severe weather,NaN,70000.0,3060.0,NaN,5348119.0,6562520,2595696.0,1.6
2,2014.0,5.0,18:38:00,Minnesota,intentional attack,vandalism,NaN,1.0,NaN,5457125.0,5284231,2640737.0,1.9
3,2010.0,10.0,20:00:00,Minnesota,severe weather,heavy wind,70000.0,3000.0,NaN,5310903.0,5222116,2586905.0,2.7
4,2012.0,6.0,04:30:00,Minnesota,severe weather,thunderstorm,68200.0,2550.0,NaN,5380443.0,5787064,2606813.0,0.6
5,2015.0,7.0,02:00:00,Minnesota,severe weather,NaN,250000.0,1740.0,250,5489594.0,5970339,2673531.0,1.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1527,2016.0,3.0,00:00:00,Idaho,intentional attack,sabotage,0.0,0.0,0,1680026.0,1657605,849763.0,2.7
1529,2016.0,7.0,15:45:00,Idaho,system operability disruption,uncontrolled loss,NaN,220.0,485,1680026.0,NaN,849763.0,2.7
1530,2011.0,12.0,08:00:00,North Dakota,public appeal,NaN,34500.0,720.0,155,685326.0,1313678,394394.0,9.8
1532,2009.0,8.0,22:54:00,South Dakota,islanding,NaN,NaN,59.0,84,807067.0,924051,436229.0,0


In [13]:
# Full plot
fig3 = px.scatter(df_no_unit, x="OUTAGE.DURATION", y="CUSTOMERS.AFFECTED", title='Scatter Plot of Number of Customers Affected vs Outage Duration')
fig3.update_layout(
    xaxis=dict(range=[0, 50000]),
    height=800)
fig3.show()

In [14]:
fig4 = px.scatter(df_no_unit, x="OUTAGE.DURATION", y="DEMAND.LOSS.MW", title='Scatter Plot of Number of Customers Affected vs Outage Duration')
fig4.update_layout(
    xaxis=dict(range=[0, 10000]),
    yaxis=dict(range=[0, 1000000]),
    height=800)
fig4.show()

In [15]:
# Scatter Plot of Number of Customers Affected vs Outage Duration Zoomed in
fig4 = px.scatter(df_no_unit, x="OUTAGE.DURATION", y="CUSTOMERS.AFFECTED", title='Scatter Plot of Number of Customers Affected vs Outage Duration')
fig4.update_layout(
    xaxis=dict(range=[0, 10000]),
    yaxis=dict(range=[0, 1000000]),
    height=600)
fig4.show()

In [16]:
df_no_nan = df_new.dropna(subset=["CUSTOMERS.AFFECTED"])

# Now group and compute the mean
agg = df_no_nan.groupby(["CAUSE.CATEGORY", "CAUSE.CATEGORY.DETAIL"])["CUSTOMERS.AFFECTED"].mean()

agg2 = df_no_nan.groupby(["CAUSE.CATEGORY", "CAUSE.CATEGORY.DETAIL"]).size()

agg = agg.to_frame().reset_index()
agg2 = agg2.to_frame("Count").reset_index()


In [17]:
agg

,CAUSE.CATEGORY,CAUSE.CATEGORY.DETAIL,CUSTOMERS.AFFECTED
0,equipment failure,breaker trip,68500.000000
1,equipment failure,cables,120000.000000
2,equipment failure,generator trip,19321.000000
3,equipment failure,line fault,30000.000000
4,equipment failure,relaying malfunction,68530.000000
5,equipment failure,substation,182000.000000
6,equipment failure,switching,100000.000000
7,equipment failure,transformer outage,18400.000000
8,equipment failure,transmission,43903.000000
9,equipment failure,transmission interruption,51500.000000


In [18]:
agg2

,CAUSE.CATEGORY,CAUSE.CATEGORY.DETAIL,Count
0,equipment failure,breaker trip,2
1,equipment failure,cables,1
2,equipment failure,generator trip,6
3,equipment failure,line fault,1
4,equipment failure,relaying malfunction,1
5,equipment failure,substation,1
6,equipment failure,switching,1
7,equipment failure,transformer outage,1
8,equipment failure,transmission,1
9,equipment failure,transmission interruption,1


In [22]:
agg_final = pd.concat([agg, agg2["Count"]], axis=1)
agg_final

,CAUSE.CATEGORY,CAUSE.CATEGORY.DETAIL,CUSTOMERS.AFFECTED,Count
0,equipment failure,breaker trip,68500.000000,2
1,equipment failure,cables,120000.000000,1
2,equipment failure,generator trip,19321.000000,6
3,equipment failure,line fault,30000.000000,1
4,equipment failure,relaying malfunction,68530.000000,1
5,equipment failure,substation,182000.000000,1
6,equipment failure,switching,100000.000000,1
7,equipment failure,transformer outage,18400.000000,1
8,equipment failure,transmission,43903.000000,1
9,equipment failure,transmission interruption,51500.000000,1


In [24]:
agg_final["SEVERE_WEATHER"] = (agg_final["CAUSE.CATEGORY"] == "severe weather").astype(int)
agg_final


,CAUSE.CATEGORY,CAUSE.CATEGORY.DETAIL,CUSTOMERS.AFFECTED,Count,SEVERE_WEATHER
0,equipment failure,breaker trip,68500.000000,2,0
1,equipment failure,cables,120000.000000,1,0
2,equipment failure,generator trip,19321.000000,6,0
3,equipment failure,line fault,30000.000000,1,0
4,equipment failure,relaying malfunction,68530.000000,1,0
5,equipment failure,substation,182000.000000,1,0
6,equipment failure,switching,100000.000000,1,0
7,equipment failure,transformer outage,18400.000000,1,0
8,equipment failure,transmission,43903.000000,1,0
9,equipment failure,transmission interruption,51500.000000,1,0


## Step 3: Assessment of Missingness

In [ ]:
# TODO

## Step 4: Hypothesis Testing

In [25]:
# Step 1: Define Observed Test Statistic
natural = agg_final[agg_final["SEVERE_WEATHER"] == 1]["CUSTOMERS.AFFECTED"]
non_natural = agg_final[agg_final["SEVERE_WEATHER"] == 0]["CUSTOMERS.AFFECTED"]

observed_diff = natural.mean() - non_natural.mean()
print("Observed difference:", observed_diff)

Observed difference: 88663.74722520796


In [27]:
# Step 2: Permutation Test
diffs = []
n_reps = 5000

# Extract original customers affected as an array
affected = agg_final["CUSTOMERS.AFFECTED"].values
labels = agg_final["SEVERE_WEATHER"].values

for _ in range(n_reps):
    # Shuffle the labels
    shuffled_labels = np.random.permutation(labels)
    
    # Compute mean difference based on shuffled labels
    diff = affected[shuffled_labels == 1].mean() - affected[shuffled_labels == 0].mean()
    diffs.append(diff)

diffs = np.array(diffs)
diffs

array([-44095.88658436,  40018.14960372, -22842.27509482, ...,
       -28387.99224654, -19318.73844002, -20947.91049744])

In [28]:
# Step 3: p-value
p_value = np.mean(diffs >= observed_diff)
print("P-value:", p_value)


P-value: 0.0006


In [29]:
# Step 4: Visualize
import pandas as pd
import plotly.express as px

df_plot = pd.DataFrame({'Diff': diffs})
fig = px.histogram(df_plot, x="Diff", nbins=50, title="Null Distribution of Mean Differences")
fig.add_vline(x=observed_diff, line_dash="dash", line_color="red", annotation_text="Observed")
fig.show()

## Step 5: Framing a Prediction Problem

In [ ]:
# TODO

## Step 6: Baseline Model

In [ ]:
# TODO

## Step 7: Final Model

In [ ]:
# TODO

## Step 8: Fairness Analysis

In [ ]:
# TODO